In [1]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [3]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

In [4]:
import pymongo
from pymongo import MongoClient
import json
from pprint import pprint
from tqdm.auto import tqdm
import re

openai_api_key  = get_secret('openai2')['api_key']

mongodb_connect = get_secret('mongodb')['connection_string']

In [5]:
mongo_client = MongoClient(mongodb_connect)
db = mongo_client.demo # use or create a database named demo
job_collection = db.job_collection #use or create a collection named tweet_collection

In [6]:
filter={

    
}
project={
    'QualificationSummary': 1, 
    'PositionID': 1
}
#rename the client to mongo_client
result = mongo_client['demo']['job_collection'].find(
  filter=filter,
  projection=project
)

In [7]:
job_data = []
#url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F]))+'
for job in result:
   # text_without_urls = re.sub(url_pattern, '', tweet['tweet']['text'])
    job_data.append({'position_id':job['PositionID'],'summary':job['QualificationSummary']})

In [8]:
print('Number of jobs: ',len(job_data))

Number of jobs:  213


In [9]:
from openai import OpenAI
client = OpenAI(api_key=openai_api_key)
model="gpt-4o"
temperature=0

def openai_help(prompt, model=model, temperature =temperature ):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature

    )
    return response.choices[0].message.content

In [10]:
for job in tqdm(job_data):
  
    prompt = f"""
    Identify the common technology skills from the following job summary,
    job summary: {job['summary']},
    format the items in a JSON list,
    be consistent, generalize, and concise,
   if no technology skills presented, use "Unknown" in the list.
    Do not wrap the JSON codes in JSON markers
   
    """
#     print(prompt)
    try:
        extract_result =openai_help(prompt)
#        print(extract_result)

        job_collection.update_one(
                {'PositionID':job['position_id']},
                {"$set":{'skills':json.loads(extract_result)}}
                )
    except:
        pass

  0%|          | 0/213 [00:00<?, ?it/s]

In [11]:
mongo_client.close()